In [1]:
import pandas as pd 
import sklearn
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from imblearn.metrics import classification_report_imbalanced

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids

In [2]:
cc_df=pd.read_csv("creditcard.csv")
#cc_df['Date'] = pd.to_datetime(cc_df['Time'],unit='s')
#cc_df.drop(columns=['Time'],axis=1)
print(cc_df)

            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  0.098698  0.363787  ... -0.01830

In [3]:
# Split into Train / Test Sets
x=cc_df.drop(columns='Class',axis=1)
print(x)

y=cc_df[['Class']].copy()
print(y)

xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=1)

            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V20       V21  \
0       0.462388  0.239599  0.098698  0.363787  ...  0.25141

In [4]:
# Oversampling (Smote) + Random Forest 
xsmote,ysmote=SMOTE(random_state=1,sampling_strategy=1.0).fit_resample(xtrain,ytrain)

print(xsmote)
print(ysmote)

                 Time         V1        V2         V3        V4        V5  \
0       159708.000000  -0.459482  1.198230   2.315423  4.274698  0.603004   
1       129139.000000   1.878563  0.020785  -1.621113  2.908813  2.507847   
2         4263.000000  -1.467414  1.210542   1.891733 -0.282189 -0.214526   
3       113041.000000   2.073474 -0.147624  -1.495803  0.184499  0.107505   
4        77352.000000   1.097497  0.357207   1.051187  2.631217 -0.464023   
...               ...        ...       ...        ...       ...       ...   
426443   41233.586829 -10.355998  6.092424 -13.081348  7.333507 -9.550066   
426444   35886.873089  -2.813944  3.771450  -3.775914  5.561832 -3.589699   
426445  143237.936649  -1.408302  2.508873  -2.686728  4.111950  0.516272   
426446  129738.321857  -1.581447  2.974258  -6.213306  3.656808 -0.810412   
426447  150719.013410  -0.261952  1.361121  -2.416682  3.613812  0.951105   

              V6         V7        V8        V9  ...       V20       V21  \

In [6]:
from sklearn.neural_network import MLPClassifier

mlpc=MLPClassifier(hidden_layer_sizes=(200,),max_iter=100)
mlpc.fit(xsmote,ysmote)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MLPClassifier(hidden_layer_sizes=(200,), max_iter=100)

In [7]:
mlpc_pred=mlpc.predict(xtest)
print(classification_report_imbalanced(ytest,mlpc_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.86      1.00      0.93      0.87     71091
          1       0.15      0.86      0.99      0.25      0.93      0.85       111

avg / total       1.00      0.99      0.87      0.99      0.93      0.87     71202



In [8]:
# Combination Sampling 
smote_enn=SMOTEENN(random_state=0)
xsmtn,ysmtn=smote_enn.fit_resample(xtrain,ytrain)

In [9]:
mlpc.fit(xsmtn,ysmtn)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(200,), max_iter=100)

In [10]:
mlpc_pred=mlpc.predict(xtest)
print(classification_report_imbalanced(ytest,mlpc_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.89      0.95      0.94      0.92      0.85     71091
          1       0.01      0.95      0.89      0.03      0.92      0.86       111

avg / total       1.00      0.89      0.95      0.94      0.92      0.85     71202



In [12]:
# Undersampling 
cc=ClusterCentroids(random_state=1)
xcc,ycc=cc.fit_resample(xtrain,ytrain)

In [13]:
mlpc.fit(xcc,ycc)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MLPClassifier(hidden_layer_sizes=(200,), max_iter=100)

In [15]:
mlpc_pred=mlpc.predict(xtest)
print(classification_report_imbalanced(ytest,mlpc_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      0.53      1.00      0.73      0.55     71091
          1       0.17      0.53      1.00      0.25      0.73      0.50       111

avg / total       1.00      1.00      0.53      1.00      0.73      0.55     71202



In [17]:
# No sampling
mlpc.fit(xtrain,ytrain.values.ravel())

MLPClassifier(hidden_layer_sizes=(200,), max_iter=100)

In [18]:
mlpc_pred=mlpc.predict(xtest)
print(classification_report_imbalanced(ytest,mlpc_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      0.13      1.00      0.36      0.14     71091
          1       0.74      0.13      1.00      0.22      0.36      0.12       111

avg / total       1.00      1.00      0.13      1.00      0.36      0.14     71202

